In [ ]:
!pip install --q git+https://github.com/m-bain/whisperx.git
!pip install -q speechbrain
!pip install flask pyngrok werkzeug flask-cors

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.7/208.7 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.5/808.5 kB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 7.5 MB/s eta 

In [ ]:
import whisperx
import gc
from IPython.display import Audio, display
import torch
import numpy as np
from speechbrain.inference.speaker import SpeakerRecognition
from google.colab import files, userdata
import subprocess
import os
import datetime

from flask import Flask, request, jsonify, send_file
from werkzeug.utils import secure_filename
from pyngrok import ngrok
from flask_cors import CORS





/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [ ]:
app = Flask(__name__)
CORS(app)
os.makedirs('uploads', exist_ok=True)
os.makedirs('results', exist_ok=True)


In [ ]:
# Initialize the SpeakerRecognition system
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")


hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [ ]:
# configurations
WHISPER_MODEL = "large-v2"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 4
compute_type = "float16" if torch.cuda.is_available() else "int8"
# compute_type = "float16"
hf_token = userdata.get('HF_TOKEN')

model = whisperx.load_model(WHISPER_MODEL, device = DEVICE , compute_type=compute_type, language = "en")


config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

100%|█████████████████████████████████████| 16.9M/16.9M [00:02<00:00, 8.58MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.3.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.3.0+cu121. Bad things might happen unless you revert torch to 1.x.


In [ ]:
diarize_model = whisperx.DiarizationPipeline(use_auth_token=hf_token,
                                             device=DEVICE)

config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

In [ ]:
# Utility functions
def ensure_wav_format(file_path):
    wav_path = os.path.splitext(file_path)[0] + '.wav'
    subprocess.call(['ffmpeg', '-i', file_path, wav_path, '-y'])
    return wav_path

def get_file_embedding(file_path):
    # signal, fs = torch.load(file_path)
    # embedding = verification.encode_batch(signal)
    # return embedding.squeeze().cpu().numpy()

    signal = verification.load_audio(file_path)
    embedding = verification.encode_batch(signal)
    return embedding.squeeze().cpu().numpy()

def time(secs):
    return datetime.timedelta(seconds=round(secs))


In [ ]:
#dictionary to store embeddings for now
known_speakers = {}

In [ ]:
#Enroll speakers with voice samples and create embeddings
@app.route('/enroll', methods=['POST'])
def enroll_speakers():
    if 'num_speakers' not in request.form:
        return jsonify({"error": "Number of speakers not provided"}), 400

    if(len(known_speakers) != 0):
       known_speakers.clear()

    num_speakers = int(request.form['num_speakers'])
    names = request.form.getlist('names')

    if len(names) != num_speakers:
        return jsonify({"error": "Number of names doesn't match number of speakers"}), 400

    if len(request.files) != num_speakers:
        return jsonify({"error": "Number of audio files doesn't match number of speakers"}), 400

    for i, name in enumerate(names):
        audio_file = request.files[f'audio_{i}']
        if audio_file:
            filename = secure_filename(audio_file.filename)
            filepath = os.path.join('uploads', filename)
            audio_file.save(filepath)
            wav_path = ensure_wav_format(filepath)
            embedding = get_file_embedding(wav_path)
            known_speakers[name] = embedding
            os.remove(filepath)
            os.remove(wav_path)

    return jsonify({"message": "Speakers enrolled successfully", "enrolled_speakers": list(known_speakers.keys())}), 200

In [ ]:
@app.route('/transcribe', methods=['POST'])
def transcribe_conversation():
    if 'audio' not in request.files:
        return jsonify({"error": "No audio file provided"}), 400

    audio_file = request.files['audio']
    filename = secure_filename(audio_file.filename)
    filepath = os.path.join('uploads', filename)
    audio_file.save(filepath)
    wav_path = ensure_wav_format(filepath)

    # 1. Transcribe
    audio = whisperx.load_audio(wav_path)
    result1 = model.transcribe(audio)
    print("result1", result1)
    print(result1["language"]) # before alignment

    # 2. Align whisper output
    model_a, metadata = whisperx.load_align_model(language_code=result1["language"], device=DEVICE)
    result2 = whisperx.align(result1["segments"], model_a, metadata, audio, DEVICE, return_char_alignments=False)
    print("result2", result2)

    # 3. Diarize
    diarize_segments = diarize_model(audio, min_speakers=len(known_speakers)) # max_speakers=len(known_speakers)
    result = whisperx.assign_word_speakers(diarize_segments, result2)
    # segments are now assigned speaker IDs

    print("Initial segments with speaker labels:")
    for segment in result["segments"]:
        print(segment.get("speaker", "No speaker"), segment)

    diarized_speakers = diarize_segments.speaker.unique()
    print("Diarized speakers:", diarized_speakers)

    speaker_mapping = {}
    available_known_speakers = list(known_speakers.keys())
    print("Available known speakers:", available_known_speakers)

    #4. Find best matching speakers for segments
    for diarized_speaker in diarized_speakers:
        speaker_segments = diarize_segments[diarize_segments.speaker == diarized_speaker]
        speaker_audio = audio[int(speaker_segments.start.iloc[0] * 16000):int(speaker_segments.end.iloc[-1] * 16000)]
        diarized_embedding = verification.encode_batch(torch.tensor(speaker_audio).unsqueeze(0)).squeeze().cpu().numpy()

        scores = {}
        for name in available_known_speakers:
            known_embedding = known_speakers[name]
            score = verification.similarity(torch.tensor(diarized_embedding).unsqueeze(0),
                                            torch.tensor(known_embedding).unsqueeze(0))
            scores[name] = score.item()

        best_match = max(scores, key=scores.get)
        speaker_mapping[diarized_speaker] = best_match
        available_known_speakers.remove(best_match)

        if not available_known_speakers:
            break

    # If there are more diarized speakers than enrolled speakers, assign generic labels
    for i, diarized_speaker in enumerate(diarized_speakers):
        if diarized_speaker not in speaker_mapping:
            speaker_mapping[diarized_speaker] = f"Unknown_Speaker_{i+1}"

    print("Speaker mapping:", speaker_mapping)

    # Replace speaker labels with identified names
    for segment in result["segments"]:
        if "speaker" in segment:
            original_speaker = segment["speaker"]
            identified_speaker = speaker_mapping.get(original_speaker, original_speaker)
            segment["speaker"] = identified_speaker
            print(f"Segment speaker changed from {original_speaker} to {identified_speaker}")

    # Debug: Print segments to verify
    print("Segments with speaker labels:")
    for segment in result["segments"]:
        print(segment)

    # Save transcription
    file_name = 'transcription' + filename + '.txt'
    transcription_path = os.path.join('results', file_name)
    with open(transcription_path, 'w') as f:
        for segment in result["segments"]:
            start_time = datetime.timedelta(seconds=round(segment['start']))
            end_time = datetime.timedelta(seconds=round(segment['end']))
            speaker = segment['speaker']
            text = segment['text']
            f.write(f"[{start_time} - {end_time}] {speaker}: {text}\n")

    print("Transcript with identified speakers saved to 'transcription.txt'")

    with open(transcription_path, 'r') as f:
      transcription_text = f.read()

    return jsonify({"transcription": transcription_text}), 200, {'Content-Type': 'application/json'}

In [ ]:
# Load Ngrok token from Colab secrets
NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
ngrok.set_auth_token(NGROK_AUTHTOKEN)
ngrok_tunnel = ngrok.connect(5000)
print(f'Public URL: {ngrok_tunnel.public_url}')
app.run()

Public URL: https://6f59-35-247-137-183.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 06:22:11] "OPTIONS /enroll HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 06:22:14] "POST /enroll HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 06:22:26] "OPTIONS /transcribe HTTP/1.1" 200 -


result1 {'segments': [{'text': " Good morning, everyone. Thank you for joining us. I'm excited to introduce two remarkable professionals today. First, we have Harsh Jain, a front-end developer at Appointee. Harsh has utilized NestJS TypeScript and Ant Design for multiple projects and created a scalable notification utility using NestJS. Outside work, Harsh enjoys conducting workshops on open source tools.", 'start': 0.009, 'end': 27.415}, {'text': ' Next we have Vanshita Savarna, a software engineer at XYZ Corp. Vanshita has led the development of innovative software solutions and is known for her expertise in cloud computing and AI. In her free time, she loves hiking and photography.', 'start': 28.456, 'end': 43.439}], 'language': 'en'}
en


Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth
100%|██████████| 360M/360M [00:01<00:00, 338MB/s]


result2 {'segments': [{'start': 0.449, 'end': 1.369, 'text': ' Good morning, everyone.', 'words': [{'word': 'Good', 'start': 0.449, 'end': 0.609, 'score': 0.908}, {'word': 'morning,', 'start': 0.649, 'end': 0.949, 'score': 0.923}, {'word': 'everyone.', 'start': 1.009, 'end': 1.369, 'score': 0.775}]}, {'start': 1.589, 'end': 2.57, 'text': 'Thank you for joining us.', 'words': [{'word': 'Thank', 'start': 1.589, 'end': 1.769, 'score': 0.995}, {'word': 'you', 'start': 1.789, 'end': 1.889, 'score': 0.841}, {'word': 'for', 'start': 1.909, 'end': 2.029, 'score': 0.785}, {'word': 'joining', 'start': 2.069, 'end': 2.39, 'score': 0.893}, {'word': 'us.', 'start': 2.45, 'end': 2.57, 'score': 0.773}]}, {'start': 3.09, 'end': 6.09, 'text': "I'm excited to introduce two remarkable professionals today.", 'words': [{'word': "I'm", 'start': 3.09, 'end': 3.23, 'score': 0.551}, {'word': 'excited', 'start': 3.29, 'end': 3.69, 'score': 0.991}, {'word': 'to', 'start': 3.75, 'end': 3.83, 'score': 0.796}, {'wo

INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 06:22:45] "POST /transcribe HTTP/1.1" 200 -


Speaker mapping: {'SPEAKER_00': 'vishal', 'SPEAKER_01': 'ankit'}
Segment speaker changed from SPEAKER_00 to vishal
Segment speaker changed from SPEAKER_00 to vishal
Segment speaker changed from SPEAKER_00 to vishal
Segment speaker changed from SPEAKER_01 to ankit
Segment speaker changed from SPEAKER_01 to ankit
Segment speaker changed from SPEAKER_01 to ankit
Segment speaker changed from SPEAKER_00 to vishal
Segment speaker changed from SPEAKER_00 to vishal
Segment speaker changed from SPEAKER_00 to vishal
Segments with speaker labels:
{'start': 0.449, 'end': 1.369, 'text': ' Good morning, everyone.', 'words': [{'word': 'Good', 'start': 0.449, 'end': 0.609, 'score': 0.908, 'speaker': 'SPEAKER_00'}, {'word': 'morning,', 'start': 0.649, 'end': 0.949, 'score': 0.923, 'speaker': 'SPEAKER_00'}, {'word': 'everyone.', 'start': 1.009, 'end': 1.369, 'score': 0.775, 'speaker': 'SPEAKER_00'}], 'speaker': 'vishal'}
{'start': 1.589, 'end': 2.57, 'text': 'Thank you for joining us.', 'words': [{'word